In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from category_encoders import BinaryEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
master = pd.read_excel('master_old.xlsx')
test = pd.read_excel('master2_old.xlsx')

In [2]:
master.drop(labels=['Ts_User_08_(Test_ID)', 'Ts_Name','Ts_TCD_Releases', 'Ts_Executing_OpCos','Scope_CZ', 'Scope_DE',
                          'Scope_HU', 'Scope_IE', 'Scope_NL','Scope_UK','Ts_TCD_Releases','date',
                     'Ts_Description(draft)','fail_count','pass_count','Cy_Executing_OpCo',
                     'pass_ratio','Rn_Tester_Name','Terminal'],axis=1,inplace=True)
test.drop(labels=['Ts_User_08_(Test_ID)', 'Ts_Name','Ts_TCD_Releases', 'Ts_Executing_OpCos','Scope_CZ', 'Scope_DE',
                          'Scope_HU', 'Scope_IE', 'Scope_NL','Scope_UK','Ts_TCD_Releases','date',
                     'Ts_Description(draft)','fail_count','pass_count','Cy_Executing_OpCo',
                     'pass_ratio','Rn_Tester_Name','Terminal'],axis=1,inplace=True)

In [3]:
master.loc[len(master)] = ['missing',0,'missing','2 - Serious','missing','missing','missing','No Run',0,0,0,'No Defect','NA',0,0,0,
               'missing','missing','missing','missing']


In [4]:
master['defect_from_RN'].fillna(0,inplace=True)
test['defect_from_RN'].fillna(0,inplace=True)
master['defect_from_RN'].replace({'missing':0}, inplace=True)

In [5]:
master['Cy_Terminal_Name'].fillna('missing',inplace=True)
master['Cy_Terminal_Release'].fillna('missing',inplace=True)
master['Cy_Terminal_SW_Version'].fillna('missing',inplace=True)
master['Scope_CTC'].fillna(0,inplace=True)
master['Ts_Suggested_Severity'].fillna('missing',inplace=True)
master['Rn_Status'].fillna('No Run',inplace=True)
master['fail_ratio'].fillna(0.053,inplace=True)
master['soft_ver_count'].fillna(0,inplace=True)
master['Scope_CTC'].replace({'X':1}, inplace=True)
master['Ts_Suggested_Severity'].replace({'4 - Minor':1, '3 - Major':2,'2 - Serious':3,'1b - Critical':4,'1a - Fatal':5,'missing':0}, inplace=True)
master['Rn_Status'].replace({'No Run':0, 'Passed':1, 'Design Limitation':2,'Blocked':3,'Failed':4}, inplace=True)
master['Defect_ID'].fillna(0,inplace=True)
master['Status'].fillna('No Defect',inplace=True)
master['Severity'].fillna('NA',inplace=True)
master['total_defect_count'].fillna(0,inplace=True)
master['open_defect_count'].fillna(0,inplace=True)

test['Cy_Terminal_Name'].fillna('missing',inplace=True)
test['Cy_Terminal_Release'].fillna('missing',inplace=True)
test['Cy_Terminal_SW_Version'].fillna('missing',inplace=True)
test['Scope_CTC'].fillna(0,inplace=True)
test['Ts_Suggested_Severity'].fillna('missing',inplace=True)
test['Rn_Status'].fillna('No Run',inplace=True)
test['fail_ratio'].fillna(0.053,inplace=True)
test['soft_ver_count'].fillna(0,inplace=True)
test['Scope_CTC'].replace({'X':1, 0:0}, inplace=True)
test['Ts_Suggested_Severity'].replace({'4 - Minor':1, '3 - Major':2,'2 - Serious':3,'1b - Critical':4,'1a - Fatal':5,'missing':0}, inplace=True)
test['Rn_Status'].replace({'No Run':0, 'Passed':1, 'Design Limitation':2,'Blocked':3,'Failed':4}, inplace=True)
test['Defect_ID'].fillna(0,inplace=True)
test['Status'].fillna('No Defect',inplace=True)
test['Severity'].fillna('NA',inplace=True)
test['total_defect_count'].fillna(0,inplace=True)
test['open_defect_count'].fillna(0,inplace=True)

In [6]:
master['Severity'].replace({'NA':0,'4 - Minor':1, '3 - Major':2,'2 - Serious':3,
                                        '1b - Critical':4,'1a - Fatal':5}, inplace=True)
master['Status'].replace({'No Defect':0, 'Closed':1, 'Design Limitation':2,'Duplicate':3,'Rejected':3,'Suspended':3,
                                        'Retest':4,'Reopen':5,'Open':5}, inplace=True)
test['Severity'].replace({'NA':0,'4 - Minor':1, '3 - Major':2,'2 - Serious':3,
                                        '1b - Critical':4,'1a - Fatal':5}, inplace=True)
test['Status'].replace({'No Defect':0, 'Closed':1, 'Design Limitation':2,'Duplicate':3,'Rejected':3,'Suspended':3,
                                        'Retest':4,'Reopen':5,'Open':5}, inplace=True)

In [7]:
# Traning Data
le = LabelEncoder()
master['Cy_Terminal_Name'] = le.fit_transform(master['Cy_Terminal_Name'])
test['Cy_Terminal_Name'] = le.transform(test['Cy_Terminal_Name'])
master['input_terminal'] = le.transform(master['input_terminal'])
test['input_terminal'] = le.transform(test['input_terminal'])

master['Cy_Terminal_Release'] = str(master['Cy_Terminal_Release'])
test['Cy_Terminal_Release'] = str(test['Cy_Terminal_Release'])
master['Cy_Terminal_Release'] = master['Cy_Terminal_Release'].apply(lambda x: 'Launch' if x.startswith('Laun') else 'MR0')
test['Cy_Terminal_Release'] = test['Cy_Terminal_Release'].apply(lambda x: 'Launch' if x.startswith('Laun') else 'MR0')
master['Cy_Terminal_Release'] = le.fit_transform(master['Cy_Terminal_Release'])
test['Cy_Terminal_Release'] = le.transform(test['Cy_Terminal_Release'])

master['input_release'] = le.fit_transform(master['input_release'])
test['input_release'] = le.transform(test['input_release'])

master['Cy_Terminal_SW_Version'] = le.fit_transform(master['Cy_Terminal_SW_Version'])
test['Cy_Terminal_SW_Version'] = le.transform(test['Cy_Terminal_SW_Version'])

master['Ts_Test_Area'] = le.fit_transform(master['Ts_Test_Area'])
test['Ts_Test_Area'] = le.transform(test['Ts_Test_Area'])
master['area_from_RN'] = le.transform(master['area_from_RN'])
test['area_from_RN'] = le.transform(test['area_from_RN'])

master['Ts_Coverage'] = le.fit_transform(master['Ts_Coverage'])
test['Ts_Coverage'] = le.transform(test['Ts_Coverage'])

master['Defect_ID'] = le.fit_transform(master['Defect_ID'])
test['Defect_ID'] = le.transform(test['Defect_ID'])
master['defect_from_RN'] = le.transform(master['defect_from_RN'])
test['defect_from_RN'] = le.transform(test['defect_from_RN'])

In [8]:
encoder = BinaryEncoder(cols=['Cy_Terminal_Name', 'Cy_Terminal_Release','Cy_Terminal_SW_Version',
       'Ts_Test_Area', 'Ts_Coverage','input_terminal', 'input_release','defect_from_RN', 'area_from_RN'])  
master  = encoder.fit_transform(master)
test  = encoder.transform(test)

In [9]:
master.isnull().any().any()
test.isnull().any().any()
master.is_selected.value_counts()

False

False

0    44612
1      207
Name: is_selected, dtype: int64

In [10]:
# not_selected = master[master['is_selected'] == 0 ]
# sample = not_selected.sample(n=474)
# master = sample.append(master[master['is_selected'] == 1 ])

In [11]:
y_train = master['is_selected']
x_train = master.drop(labels='is_selected',axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=123)
Y_test = test['is_selected']
X_test = test.drop(labels='is_selected',axis=1)

In [12]:
randforest = RandomForestClassifier()
randforest.fit(x_train,y_train);
Y_pred_RF = randforest.predict(X_test)
y_pred_RF = randforest.predict(x_test)
accuracy_score(y_test, y_pred_RF)
f1_score(y_test, y_pred_RF) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

0.9938271604938271

0.22429906542056074

In [13]:
accuracy_score(Y_test, Y_pred_RF)
f1_score(Y_test, Y_pred_RF) 

0.9929938863849346

0.3938223938223938

In [14]:
# confusion matrix
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix
# cm_log = confusion_matrix(Y_test, Y_pred_RF)
# class_label = ["negative", "positive"]
# df_cm = pd.DataFrame(cm_log, index = class_label, columns = class_label)
# sns.heatmap(df_cm, annot = True, fmt = "d")
# plt.title("Confusion Matrix")
# plt.xlabel("Prediction Label")
# plt.ylabel("Truth Label")
# plt.show()

In [15]:
# feature_importances = pd.DataFrame(randforest.feature_importances_,
#                                    index = x_train.columns,
#                                     columns=['importance']).sort_values('importance',ascending=False)
# feature_importances

## PCA

In [16]:
from sklearn.decomposition import PCA
pca_again = PCA(0.99)
xtrain_pca2 = pca_again.fit_transform(x_train,y_train)
Xtest_pca2 = pca_again.transform(X_test)
xtest_pca2 = pca_again.transform(x_test)

In [17]:
randforest.fit(xtrain_pca2,y_train);
Y_pred_RF = randforest.predict(Xtest_pca2)
y_pred_RF = randforest.predict(xtest_pca2)
accuracy_score(y_test, y_pred_RF)
f1_score(y_test, y_pred_RF) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

0.9933809311319352

0.2644628099173553

In [18]:
accuracy_score(Y_test, Y_pred_RF)
f1_score(Y_test, Y_pred_RF) 

0.994198759427016

0.548611111111111

In [40]:
# stratified k fold
randforest = RandomForestClassifier(n_jobs=-1,random_state=123,class_weight='balanced')
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 123)
param = {
    "n_estimators": [5,10,15,20,40,60,80,120], "max_depth": [3, 5,7,11,15,21,30,40,50]
}
clf = GridSearchCV(randforest,param,cv=folds)
clf.fit(x_train,y_train)
clf.best_params_
Y_pred_RF = clf.predict(X_test)
accuracy_score(Y_test, Y_pred_RF)
f1_score(Y_test, Y_pred_RF) 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
       error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators='warn', n_jobs=-1, oob_score=False,
            random_state=123, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [5, 10, 15, 20, 40, 60, 80, 120], 'max_depth': [3, 5, 7, 11, 15, 21, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'max_depth': 30, 'n_estimators': 5}

0.997367129278415

0.8467532467532468

In [20]:
# stratified k fold
clf.fit(xtrain_pca2,y_train)
clf.best_params_
Y_pred_RF = clf.predict(xtest_pca2)
accuracy_score(y_test, Y_pred_RF)
f1_score(y_test, Y_pred_RF) 
Y_pred_RF = clf.predict(Xtest_pca2)
accuracy_score(Y_test, Y_pred_RF)
f1_score(Y_test, Y_pred_RF) 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
       error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators='warn', n_jobs=-1, oob_score=False,
            random_state=123, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [5, 10, 15, 20, 40, 60, 80, 120], 'max_depth': [3, 5, 7, 11, 15, 21, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

{'max_depth': 30, 'n_estimators': 10}

0.9933809311319352

0.4606060606060607

0.9970993797135079

0.8320413436692508

In [21]:
def join_rows(df) :
    new_df = pd.DataFrame(index=range(1),columns = df.columns)
    for i in range(df.shape[1]) :                       
        temp = set()
        for j in range(df.shape[0]) :
            if j == 0:
                new_df.iloc[j,i] = df.iloc[j,i]
            else :
                if df.iloc[j,i] != new_df.iloc[0,i] :
                    temp.add(str(df.iloc[j,i]))
        if (len(temp) > 0) :
            temp.add(str(new_df.iloc[0,i]))
            if 'nan' in temp :
                temp.remove('nan')
            new_df.iloc[0,i] = ', '.join(temp)
    return new_df

In [41]:
test = pd.read_excel('master2_old.xlsx')
test['is_predicted'] = Y_pred_RF
#test['Reason to select'] = ''
test.loc[test['is_predicted']==1,'Reason to select'] = 'model prediction'
test.loc[test['is_predicted']==0,'Reason to select'] = 'NA'

In [42]:
frame =  test[test['is_predicted']==1]

In [43]:
group = frame.groupby(['Ts_Name'])
final = pd.DataFrame()
final = final.append(group.apply(join_rows))
#final.to_excel('frame.xlsx',index=False)

In [44]:
frame = final.copy()

In [45]:
rule = pd.read_excel('FinalTestSuite_nonOptimized.xlsx')  #142  224
rule['is_predicted'] = 1
rule['is_selected'] = 1

In [46]:
#rule.columns
rule.shape
rule['Ts_Name'].isna().any()

(141, 28)

False

In [47]:
#frame.columns
frame.shape
frame['Ts_Name'].isna().any()

(163, 40)

False

a = np.intersect1d(frame.columns, rule.columns)
a

In [48]:
join = frame.append(rule)
join.shape
group = join.groupby(['Ts_Name'])
final = pd.DataFrame()
final = final.append(group.apply(join_rows))
final.shape

(304, 44)

(279, 44)

In [49]:
final1 = final.copy()
final1['Ts_Suggested_Severity'].unique()

array(['1b - Critical', '1a - Fatal', '2 - Serious', '3 - Major'],
      dtype=object)

## Criteria One for optimazation ( release note specific )

In [50]:
final = final1.copy()
final.shape
final = final[((final['Rn_Status'] != 'Passed') & (final['Reason to select'].str.contains('Change in version/status of test area')))  |
             (final['Reason to select'].str.contains('linked TC for defect id:'))]
final.shape
#(12, 44)

(279, 44)

(63, 44)

In [32]:
# final = final1.copy()
# final.shape
# final = final[(((final['Ts_Suggested_Severity'] == '1a - Fatal') | (final['Ts_Suggested_Severity'] ==  '1b - Critical')) & 
#                 (final['Reason to select'].str.contains('Change in version/status of test area')))  | 
#               (((final['Ts_Suggested_Severity'] == '1a - Fatal') | (final['Ts_Suggested_Severity'] ==  '1b - Critical')) & 
#                (final['Reason to select'].str.contains('model prediction'))) |
#               (final['Reason to select'].str.contains('linked TC for defect id:')) | 
#                ( final['Rn_Status'].str.contains('Failed')) ]
# final.shape

In [33]:
# final = final1.copy()
# final.shape
# final = final[(((final['Rn_Status'].str.contains('Failed')) ) & 
#                 (final['Reason to select'].str.contains('Change in version/status of test area')))  | 
#               (((final['Ts_Suggested_Severity'] == '1a - Fatal') | (final['Ts_Suggested_Severity'] ==  '1b - Critical')) & 
#                (final['Reason to select'].str.contains('model prediction'))) |
#               (final['Reason to select'].str.contains('linked TC for defect id:')) | 
#                ( final['Rn_Status'].str.contains('Failed')) ]
# final.shape

## Criteria Two for optimazation 

In [51]:
final = final1.copy()
final.shape
final = final[(((final['Cy_Terminal_Name'].str.contains('Hima')) ) & (final['input_terminal'] == 'Hima') 
               & (final['Ts_Suggested_Severity'] == '1a - Fatal') &
                (final['Reason to select'].str.contains('Change in version/status of test area')))  |
                
              (((final['Ts_Suggested_Severity'] == '1a - Fatal') ) & 
               (final['Reason to select'].str.contains('model prediction'))) |
              (final['Reason to select'].str.contains('linked TC for defect id:')) | 
               ( final['Rn_Status'].str.contains('Failed')) ]
final.shape
#Cy_Terminal_Name Reason to select  input_terminal

(279, 44)

(166, 44)

final.is_predicted.value_counts()

test = pd.read_excel('master2_old.xlsx')


final.to_excel('final.xlsx',index=False)

In [35]:
# inner = pd.merge(frame,rule,how='inner',on='Ts_Name')
# inner.shape
# (82, 67)
# list of columns that can be deleted directly  

inner.to_excel('inner.xlsx',index=False)

inner[inner['Defect_ID_x'] != inner['Defect_ID_y']]

In [36]:
#inner[inner['Cy_Terminal_SW_Version_x'] != inner['Cy_Terminal_SW_Version_y']][['Cy_Terminal_SW_Version_x','Cy_Terminal_SW_Version_y']]

In [37]:
# out = pd.merge(frame,rule,how='outer',on='Ts_Name')
# out.shape

In [38]:
#out.to_excel('out.xlsx',index=False)

In [39]:
# final['fail_ratio'].fillna(0,inplace=True)
# final['fail_ratio'].replace({'':0},inplace=True)
# final['fail_ratio'] = final['fail_ratio'].astype(float)
# final['fail_ratio'].dtypes